# This notebook shows how CpG net can be used on real data. A full pipeline run is included. This notebook can easily be adapted into a script that can be run on  a compute cluster.

Jack Duryea  
Computational Epigenetics Section  
Waterland Lab  
Baylor College of Medicine  
May 2018



In [1]:
from CpG_Net import CpGNet
from CpG_Bin import Bin
import numpy as np
import cPickle as pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from random import shuffle
%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## 1. Load data

In [ ]:
data = pickle.load(open("HAMbins.p","rb")) 

In [ ]:
print "There are ", len(data) , " bins in the dataset"

## 2. Filter read depth (at least 20 reads)

In [ ]:
min_read_depth = 20
read_filtered_data = [bin_ for bin_ in data if bin_.matrix.shape[0] >= min_read_depth]

In [ ]:
print "There are ", len(read_filtered_data), " bins that meet the read depth requirement"

## 3. Split by density, up to 8Cpgs in 100 bp bins

In [ ]:
cpg_2_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==2]
cpg_3_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==3]
cpg_4_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==4]
cpg_5_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==5]
cpg_6_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==6]
cpg_7_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==7]
cpg_8_bins = [bin_ for bin_ in read_filtered_data if bin_.matrix.shape[1]==8]

In [ ]:
# get a subset of the data to speed things up
shuffle(cpg_2_bins)
cpg_2_bins_subset = cpg_2_bins[:10000]

# 4. Create CpG Net (this example is for 2 CpGs)

In [ ]:
net = CpGNet(cpgDensity=2)

# 5. Collect Feature Vectors

In [ ]:
X, y = net.collectFeatures(cpg_2_bins_subset) # extract features

In [ ]:
X[1000]

In [ ]:
# filter out missing values
nonneg = y!=-1
X_u = X[nonneg]
y_u = y[nonneg]

# 6. Train/Test Split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_u, y_u)

In [ ]:
plt.hist(y_train)

In [ ]:
print np.count_nonzero(y_test==1)/float(len(y_test))
print np.count_nonzero(y_train==1)/float(len(y_train))

## Preprocess (very important!)

In [ ]:
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

In [ ]:
print X_train_scaled.shape
print y_train.shape

In [ ]:
history = net.fit(X_train_scaled, y_train, val_split=0.2, epochs=10000, batch_size=16)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [ ]:
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])

In [ ]:
y_pred = net.model.predict(X_test_scaled)


In [ ]:
y_pred_class = np.round(y_pred)


In [ ]:
np.sum(y_pred_class==y_test)

In [ ]:
y_pred_class

In [ ]:
plt.hist(y_pred,bins=50);

In [ ]:
plt.hist(y_train)

# PCA

In [ ]:
from sklearn.decomposition import PCA
#pca = PCA(n_components=X_train_u_scaled.shape[1])
pca = PCA(n_components=11)


In [ ]:
pca.fit(X_train_u_scaled)

In [ ]:
print pca.explained_variance_ratio_

In [ ]:
X_train_u_scaled_pca = pca.transform(X_train_u_scaled)

In [ ]:
X_train_u_scaled_pca.shape

In [ ]:
# Logistic regression
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)
logreg.score(X_train_scaled,y_train)

In [ ]:
logreg.score(X_test_scaled, y_test)

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_scaled, y_train)

In [ ]:
rf.score(X_test_scaled, y_test)

In [ ]:
rf.score(X_train_scaled, y_train)